# Excitations on a tree geometry

In [1]:
using ITensors
using ITensorNetworks
using Random

In this notebook we characterize excitations for a spin model on a lattice with a tree-like geometry using the [DMRG-X algorithm](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.116.247204).

As an example, we will consider a random field spin-1/2 Heisenberg model with Hamiltonian

$$
H = \sum_{\langle i, j \rangle} \vec{S}_i \cdot \vec{S}_j + \sum_i h_i S^z_i,
$$

on a spin system defined on a small comb tree with 6 sites
<center><img align="center" width="200" src="./fig/small_comb_tree.svg"></center>

In [2]:
c = named_comb_tree((3, 2))
s = siteinds("S=1/2", c)
root_vertex = (1, 2)

Random.seed!(12)
W = 12
# random fields h ∈ [-W, W]
h = W * (2 * rand(nv(c)) .- 1)
H = TTN(ITensorNetworks.heisenberg(c; h), s);

With the DMRG-X algorithm, we can now find the excited state of the given Hamiltonian that is closest to a given initial state. As an example, we choose the initial state to be a product state with random up/down spins at each site, and run a 2-site version of the DMRG-X algorithm:

In [3]:
ψ = normalize!(TTN(s, v -> rand(["↑", "↓"])))

dmrg_x_kwargs = (
    nsweeps=20, reverse_step=false, normalize=true, maxdim=20, cutoff=1e-10, outputlevel=0
)

ϕ = dmrg_x(H, ψ; nsite=2, dmrg_x_kwargs...)

@show abs(inner(ψ', H, ψ) / inner(ψ, ψ) - inner(ϕ', H, ϕ) / inner(ϕ, ϕ)) < 1e-1
@show abs(inner(H, ψ, H, ψ) - inner(ψ', H, ψ)^2) > 1e-2
@show abs(inner(H, ϕ, H, ϕ) - inner(ϕ', H, ϕ)^2) < 1e-7;

abs(inner(ψ', H, ψ) / inner(ψ, ψ) - inner(ϕ', H, ϕ) / inner(ϕ, ϕ)) < 0.1 = true
abs(inner(H, ψ, H, ψ) - inner(ψ', H, ψ) ^ 2) > 0.01 = true
abs(inner(H, ϕ, H, ϕ) - inner(ϕ', H, ϕ) ^ 2) < 1.0e-7 = true


For this small system, we can compare our result to exact diagonalization to see if DMRG-X gave the expected result:

In [4]:
@disable_warn_order U0 = contract(ψ, root_vertex)
@disable_warn_order T = contract(H, root_vertex)
D, U = eigen(T; ishermitian=true)
u = uniqueind(U, T)
_, max_ind = findmax(abs, array(dag(U0) * U))
U_exact = U * dag(onehot(u => max_ind))
@disable_warn_order U_dmrgx = contract(ϕ, root_vertex)
@show abs(inner(ϕ', H, ϕ) - (dag(U_exact') * T * U_exact)[]) < 1e-6
@show abs(abs(inner(U_dmrgx, U_exact)) - 1) < 1e-6;

abs(inner(ϕ', H, ϕ) - (dag(U_exact') * T * U_exact)[]) < 1.0e-6 = true
abs(abs(inner(U_dmrgx, U_exact)) - 1) < 1.0e-6 = true
